In [24]:
import requests
from bs4 import BeautifulSoup
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

# URL ของแหล่งข้อมูลใหม่ (Tilda Help Center)
url = "https://www.tilda.com/faqs/"

# ส่ง HTTP GET request ไปยัง URL
print(f"กำลังดึงข้อมูลจากหน้า How-to: {url}")
response = requests.get(url)
response.raise_for_status()  # เช็คว่า request สำเร็จหรือไม่

# ใช้ BeautifulSoup ในการ parse HTML
soup = BeautifulSoup(response.content, 'html.parser')
# จากการตรวจสอบหน้าเว็บใหม่ เนื้อหาหลักจะอยู่ใน <div id="allrecords">
# เราจะดึงข้อความจากส่วนนี้แทนที่ของเดิม
main_content = soup.find('div', class_='faqs-list')

if main_content:
    # ดึงข้อความทั้งหมดและทำความสะอาดเล็กน้อย
    # separator='\n' ช่วยให้แต่ละบรรทัดแยกจากกัน ทำให้แบ่ง chunk ได้ดีขึ้น
    document_text = main_content.get_text(separator='\n', strip=True)
    print("✅ ดึงข้อมูลจากหน้า How-to สำเร็จแล้ว")
    # print("\n--- ตัวอย่างข้อมูลที่ดึงได้ ---")
    # print(document_text[:500]) # แสดงผล 500 ตัวอักษรแรก
else:
    raise ValueError("ไม่พบเนื้อหาส่วน <div id='allrecords'> ในหน้าที่ระบุ")
print(document_text[:500])

กำลังดึงข้อมูลจากหน้า How-to: https://www.tilda.com/faqs/
✅ ดึงข้อมูลจากหน้า How-to สำเร็จแล้ว


# สร้าง instance ของ Text Splitter

In [29]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,  # ขนาดของแต่ละ chunk (ตัวอักษร)
    chunk_overlap=50, # ขนาดของส่วนที่คาบเกี่ยวกันระหว่าง chunk
    length_function=len,
)


กำลังแบ่งข้อมูลเป็นส่วนย่อย (Chunking)...
✅ แบ่งข้อมูลออกเป็น 73 chunks สำเร็จ
Company
Is Tilda an Indian Company?
Tilda is a British company that was founded by a Ugandan family who migrated to the UK back in the ’70s. Tracing back to its Indian roots, the Thakrar family wanted to bring authentic ingredients
family wanted to bring authentic ingredients they loved back home to the table for families to enjoy. 50 years on, Tilda strongly remains to be a household brand amongst different communities.


# ทำการแบ่งเอกสาร

In [ ]:
print("\nกำลังแบ่งข้อมูลเป็นส่วนย่อย (Chunking)...")
chunks = text_splitter.split_text(document_text)
print(f"✅ แบ่งข้อมูลออกเป็น {len(chunks)} chunks สำเร็จ")

# print("\n--- ตัวอย่าง Chunk แรก ---")
print(chunks[0])
print(chunks[1])
print(chunks[2])

# ขั้นตอนที่ 1: สร้าง Embeddings

In [ ]:
# สมมติว่าคุณมีตัวแปร 'chunks' ที่เป็น list ของข้อความที่แบ่งแล้ว
# chunks = ["Company", "Is Tilda an Indian Company?", "Tilda is a British company...", ...]

# เราต้องแปลงข้อความแต่ละ chunk ให้เป็น Vector ที่สื่อความหมาย
# เราจะใช้โมเดลจาก OpenAI ในการทำสิ่งนี้

from langchain_openai import OpenAIEmbeddings

import os
os.environ["OPENAI_API_KEY"] = "sk-xxxxxxxxxx"

print("\nกำลังสร้าง Embedding Model...")
embedding_model = OpenAIEmbeddings()
print("✅ Embedding Model พร้อมใช้งาน")

# ขั้นตอนที่ 2: จัดเก็บลงใน Vector Store (ChromaDB)

In [ ]:
# นำข้อความ (chunks) พร้อมกับ Embedding ที่สร้างขึ้น ไปเก็บใน ChromaDB
# ChromaDB จะทำหน้าที่เป็นฐานข้อมูลที่ให้เราค้นหาความหมายที่ใกล้เคียงได้

from langchain_community.vectorstores import Chroma

print("\nกำลังสร้าง Vector Store ด้วย ChromaDB...")
# LangChain จะทำการ Embed และจัดเก็บข้อมูลลงใน ChromaDB ให้โดยอัตโนมัติ
# เราใช้ .from_texts เพราะว่า 'chunks' ของเราเป็น list ของ string
vectorstore = Chroma.from_texts(texts=chunks, embedding=embedding_model)
print("✅ Vector Store สร้างเสร็จสมบูรณ์")

# ขั้นตอนที่ 3: สร้าง RAG Chain เพื่อถาม-ตอบ

In [33]:
# ส่วนนี้คือการประกอบร่างระบบทั้งหมดเข้าด้วยกัน
# 1. Retriever: ตัวค้นหาข้อมูลจาก Vector Store
# 2. LLM: ตัวสร้างคำตอบ (เราจะใช้ GPT)
# 3. Prompt: คำสั่งที่บอกให้ LLM ตอบคำถามโดยอิงจากข้อมูลที่ Retriever ค้นเจอ

from langchain_openai import ChatOpenAI
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

print("\nกำลังสร้าง RAG Chain...")

# 3.1) กำหนด LLM ที่จะใช้
llm = ChatOpenAI(model="gpt-4o-mini")

# 3.2) สร้าง Retriever จาก Vector Store ของเรา
retriever = vectorstore.as_retriever()

# 3.3) สร้าง Prompt Template เพื่อสั่งให้ LLM ตอบจาก context ที่ให้ไป
prompt = ChatPromptTemplate.from_template("""
ตอบคำถามต่อไปนี้โดยอ้างอิงจากบริบท (context) ที่ให้มาเท่านั้น

<context>
{context}
</context>

คำถาม: {input}
""")

# 3.4) สร้าง Chain ที่เชื่อมทุกอย่างเข้าด้วยกัน
document_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, document_chain)

print("✅ RAG Chain พร้อมสำหรับถามคำถาม!")


กำลังสร้าง Embedding Model...
✅ Embedding Model พร้อมใช้งาน

กำลังสร้าง Vector Store ด้วย ChromaDB...
✅ Vector Store สร้างเสร็จสมบูรณ์

กำลังสร้าง RAG Chain...
✅ RAG Chain พร้อมสำหรับถามคำถาม!


# ขั้นตอนที่ 4: ทดลองถามคำถาม

In [34]:
# ลองถามคำถามที่คิดว่ามีคำตอบอยู่ในข้อมูลที่เราดึงมา

question = "Is Tilda an Indian company?" # หรือ "ทิลด้าเป็นบริษัทของอินเดียหรือไม่"

print(f"\n❓ คำถาม: {question}")

# เรียกใช้ Chain เพื่อหาคำตอบ
response = rag_chain.invoke({"input": question})

# แสดงผลลัพธ์
print("\n💡 คำตอบ:")
print(response["answer"])

# (ตัวเลือกเสริม) ดูว่าระบบไปค้นเจอข้อมูลส่วนไหนมาใช้ตอบ
print("\n--- ข้อมูลอ้างอิง (Context) ---")
for doc in response["context"]:
    print(f"- {doc.page_content}")


❓ คำถาม: Is Tilda an Indian company?

💡 คำตอบ:
No, Tilda is a British company.

--- ข้อมูลอ้างอิง (Context) ---
- Company
Is Tilda an Indian Company?
- Tilda is a British company that was founded by a Ugandan family who migrated to the UK back in the ’70s. Tracing back to its Indian roots, the Thakrar family wanted to bring authentic ingredients
- What does Tilda stand for?
Tilda is the amalgamation of the names Tila and Daksha, the daughters of the founders. Learn more about Tilda’s
history
- family wanted to bring authentic ingredients they loved back home to the table for families to enjoy. 50 years on, Tilda strongly remains to be a household brand amongst different communities.
